In [0]:
spark.conf.set(
    "fs.azure.account.key.hexaolympicdl.dfs.core.windows.net","inEeVMZmBhP4rl30EO+heeCEKJ6mcwgjuugZEbF8JF6z0xgJRZdK4/iHKV7MjX5RqmTDYiPhgCM8+AStc7V2uw==")

In [0]:
dbutils.fs.ls("abfss://input@hexaolympicdl.dfs.core.windows.net/")

[FileInfo(path='abfss://input@hexaolympicdl.dfs.core.windows.net/DIM.Date.Table.csv', name='DIM.Date.Table.csv', size=796835, modificationTime=1751618665000),
 FileInfo(path='abfss://input@hexaolympicdl.dfs.core.windows.net/OlympicsAnalysis_Source1.csv', name='OlympicsAnalysis_Source1.csv', size=6249697, modificationTime=1751618665000),
 FileInfo(path='abfss://input@hexaolympicdl.dfs.core.windows.net/OlympicsAnalysis_Source2.csv', name='OlympicsAnalysis_Source2.csv', size=3299, modificationTime=1751618665000)]

In [0]:
storage_account_name = "hexaolympicdl"
container_name = "input"
storage_account_access_key = "inEeVMZmBhP4rl30EO+heeCEKJ6mcwgjuugZEbF8JF6z0xgJRZdK4/iHKV7MjX5RqmTDYiPhgCM8+AStc7V2uw=="

# This is the mount point in DBFS
mount_point = f"/mnt/{container_name}"
source_uri = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net"

# Mount it
dbutils.fs.mount(
  source = source_uri,
  mount_point = mount_point,
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_access_key}
)


True

In [0]:
dbutils.fs.ls('/mnt/input')

[FileInfo(path='dbfs:/mnt/input/DIM.Date.Table.csv', name='DIM.Date.Table.csv', size=796835, modificationTime=1751618665000),
 FileInfo(path='dbfs:/mnt/input/OlympicsAnalysis_Source1.csv', name='OlympicsAnalysis_Source1.csv', size=6249697, modificationTime=1751618665000),
 FileInfo(path='dbfs:/mnt/input/OlympicsAnalysis_Source2.csv', name='OlympicsAnalysis_Source2.csv', size=3299, modificationTime=1751618665000)]

In [0]:
df=spark.read.csv('dbfs:/mnt/input/OlympicsAnalysis_Source1.csv',inferSchema=True,header=True)

In [0]:
df.limit(10).display()

Player_Name Player_Gender Player_Age Player_Height Player_Weight Country_Name Country_id Olympic_Name Olympic_Year Olympic_Season Olympic_City Sport_Name Event_Name Medal Aaron Arthur Cook M 25 183 80.0 Moldova MDA 2016 Summer 2016 Summer Rio de Janeiro Taekwondo Taekwondo Men's Welterweight NA Aaron Brown M 24 198 79.0 Canada CAN 2016 Summer 2016 Summer Rio de Janeiro Athletics Athletics Men's 100 metres NA Aaron Brown M 24 198 79.0 Canada CAN 2016 Summer 2016 Summer Rio de Janeiro Athletics Athletics Men's 200 metres NA Aaron Brown M 24 198 79.0 Canada CAN 2016 Summer 2016 Summer Rio de Janeiro Athletics Athletics Men's 4 x 100 metres Relay Bronze Aaron Gate M 25 181 71.0 New Zealand NZL 2016 Summer 2016 Summer Rio de Janeiro Cycling Cycling Men's Team Pursuit, 4,000 metres NA Aaron John Royle M 26 180 67.0 Australia AUS 2016 Summer 2016 Summer Rio de Janeiro Triathlon Triathlon Men's Olympic Distance NA Aaron Russell M 23 206 93.0 United States USA 2016 Summer 2016 Summer Rio de Janeiro Volleyball Volleyball Men's Volleyball Bronze Aaron Younger M 24 193 100.0 Australia AUS 2016 Summer 2016 Summer Rio de Janeiro Water Polo Water Polo Men's Water Polo NA Aauri Lorena Bokesa Abia F 27 180 62.0 Spain ESP 2016 Summer 2016 Summer Rio de Janeiro Athletics Athletics Women's 400 metres NA aba Silai M 25 185 77.0 Serbia SRB 2016 Summer 2016 Summer Rio de Janeiro Swimming Swimming Men's 100 metres Breaststroke NA

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
player_dim = df.select("Player_Name", "Player_Gender", "Player_Age", "Player_Height").distinct()
windowSpec = Window.orderBy("Player_Name")
player_dim = player_dim.withColumn("Player_ID", dense_rank().over(windowSpec))
player_dim.limit(10).display()
player_dim.write.mode("overwrite").csv("abfss://dimensions@hexaolympicdl.dfs.core.windows.net/Player_details")

Player_Name Player_Gender Player_Age Player_Height Player_ID Th Ngn Thng F 23 147 1 Th Ngn Thng F 19 147 1 A Lamusi M 23 170 2 A. Joshua Josh West M 31 207 3 Aadam Ismaeel Khamis M 19 172 4 Aarik Wilson M 25 191 5 Aarn Sarmiento Padilla M 25 180 6 Aarn Sarmiento Padilla M 21 180 6 Aaron Arthur Cook M 25 183 7 Aaron Arthur Cook M 17 183 7

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
dim_sport = df.select("Sport_Name").distinct()
sport_window = Window.orderBy("Sport_Name")
dim_sport = dim_sport.withColumn("Sport_ID", dense_rank().over(sport_window))
dim_sport = dim_sport.select("Sport_ID", "Sport_Name")
dim_sport.limit(10).display()
dim_sport.write.mode("overwrite").csv("abfss://dimensions@hexaolympicdl.dfs.core.windows.net/Sport_details")

Sport_ID Sport_Name 1 Alpine Skiing 2 Archery 3 Athletics 4 Badminton 5 Baseball 6 Basketball 7 Beach Volleyball 8 Biathlon 9 Bobsleigh 10 Boxing

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
event_temp = df.select("Event_Name", "Sport_Name").distinct()
dim_event = event_temp.join(dim_sport, on="Sport_Name", how="left")
event_window = Window.orderBy("Event_Name")
dim_event = dim_event.withColumn("Event_ID", dense_rank().over(event_window))
dim_event = dim_event.select("Event_ID", "Event_Name", "Sport_ID")
dim_event.limit(10).display()
dim_event.write.mode("overwrite").csv("abfss://dimensions@hexaolympicdl.dfs.core.windows.net/Event_details")

Event_ID Event_Name Sport_ID 1 Alpine Skiing Men's Combined 1 2 Alpine Skiing Men's Downhill 1 3 Alpine Skiing Men's Giant Slalom 1 4 Alpine Skiing Men's Slalom 1 5 Alpine Skiing Men's Super G 1 6 Alpine Skiing Women's Combined 1 7 Alpine Skiing Women's Downhill 1 8 Alpine Skiing Women's Giant Slalom 1 9 Alpine Skiing Women's Slalom 1 10 Alpine Skiing Women's Super G 1

In [0]:
from pyspark.sql.functions import rank
from pyspark.sql.window import Window

# Extract distinct country data
dim_country = df.select("Country_id", "Country_Name").distinct()


dim_country.limit(10).display()
dim_country.write.mode("overwrite").csv("abfss://dimensions@hexaolympicdl.dfs.core.windows.net/Country_details")

Country_id Country_Name UZB Uzbekistan SRB Serbia ETH Ethiopia IOA Individual Olympic Athletes ESP Spain USA United States CAN Canada AUS Australia MDA Moldova NZL New Zealand

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
dim_olympic=df.select("Olympic_Name","Olympic_Year","Olympic_Season","Olympic_City").distinct()
window_spec=Window.orderBy("Olympic_Name")
dim_olympic=dim_olympic.withColumn("Olympic_id",dense_rank().over(window_spec))
dim_olympic.display()
dim_olympic.write.mode("overwrite").csv("abfss://dimensions@hexaolympicdl.dfs.core.windows.net/Olympic_details")

Olympic_Name Olympic_Year Olympic_Season Olympic_City Olympic_id 2008 Summer 2008 Summer Beijing 1 2010 Winter 2010 Winter Vancouver 2 2012 Summer 2012 Summer London 3 2014 Winter 2014 Winter Sochi 4 2016 Summer 2016 Summer Rio de Janeiro 5

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
dim_medal=df.select("Medal").distinct()
window_spec=Window.orderBy("Medal")
dim_medal=dim_medal.withColumn("Medal_id",dense_rank().over(window_spec))
dim_medal.display()
dim_medal.write.mode("overwrite").csv("abfss://dimensions@hexaolympicdl.dfs.core.windows.net/Medal_details")

Medal Medal_id Bronze 1 Gold 2 NA 3 Silver 4

In [0]:
df1_alias = df.alias("src")
dim_player_alias = player_dim.alias("p")
dim_country_alias = dim_country.alias("c")
dim_olympic_alias = dim_olympic.alias("o")
dim_sport_alias = dim_sport.alias("s")
dim_event_alias = dim_event.alias("e")
dim_medal_alias = dim_medal.alias("m")

In [0]:
fact_df = df1_alias \
    .join(dim_player_alias, on=[
        col("src.Player_Name") == col("p.Player_Name"),
        col("src.Player_Gender") == col("p.Player_Gender"),
        col("src.Player_Age") == col("p.Player_Age"),
        col("src.Player_Height") == col("p.Player_Height")
    ], how="left") \
    .join(dim_country_alias, on=col("src.Country_Name") == col("c.Country_Name"), how="left") \
    .join(dim_olympic_alias, on=[
        col("src.Olympic_Name") == col("o.Olympic_Name"),
        col("src.Olympic_Year") == col("o.Olympic_Year"),
        col("src.Olympic_Season") == col("o.Olympic_Season"),
        col("src.Olympic_City") == col("o.Olympic_City")
    ], how="left") \
    .join(dim_sport_alias, on=col("src.Sport_Name") == col("s.Sport_Name"), how="left") \
    .join(dim_event_alias, on=[
        col("src.Event_Name") == col("e.Event_Name"),
        col("s.Sport_ID") == col("e.Sport_ID")
    ], how="left") \
    .join(dim_medal_alias, on=col("src.Medal") == col("m.Medal"), how="left")

fact_df_=fact_df.select(
    "src.Player_Name", "p.Player_ID", "src.Player_Height", "src.Player_Weight",
    "c.Country_ID", "o.Olympic_ID", "e.Event_ID", "s.Sport_ID", "m.Medal_ID"
)
fact_df_.limit(10).display()
fact_df_.write.mode("overwrite").csv("abfss://facts@hexaolympicdl.dfs.core.windows.net/Player_details_facts")


Player_Name Player_ID Player_Height Player_Weight Country_ID Olympic_ID Event_ID Sport_ID Medal_ID Aaron Arthur Cook 7 183 80.0 MDA 5 372 44 3 Aaron Brown 9 198 79.0 CAN 5 17 3 3 Aaron Brown 9 198 79.0 CAN 5 20 3 3 Aaron Brown 9 198 79.0 CAN 5 22 3 1 Aaron Gate 11 181 71.0 NZL 5 144 14 3 Aaron John Royle 14 180 67.0 AUS 5 384 47 3 Aaron Russell 17 206 93.0 USA 5 386 48 1 Aaron Younger 19 193 100.0 AUS 5 388 49 3 Aauri Lorena Bokesa Abia 20 180 62.0 ESP 5 48 3 3 aba Silai 28946 185 77.0 SRB 5 332 41 3

In [0]:
from pyspark.sql.functions import col


country_fact_df = df1_alias.alias("src") \
    .join(dim_country_alias.alias("c"), col("src.Country_Name") == col("c.Country_Name"), "left") \
    .join(dim_olympic_alias.alias("o"),
          (col("src.Olympic_Name") == col("o.Olympic_Name")) &
          (col("src.Olympic_Year") == col("o.Olympic_Year")) &
          (col("src.Olympic_Season") == col("o.Olympic_Season")) &
          (col("src.Olympic_City") == col("o.Olympic_City")),
          "left")
from pyspark.sql.functions import countDistinct, count, when, col, round


fact_country_details = country_fact_df.groupBy("c.Country_ID", "o.Olympic_ID", "o.Olympic_Year", "c.Country_Name").agg(
    count(when(col("src.Medal").isNotNull(), True)).alias("Total_No_Of_Medals"),
    count(when(col("src.Medal") == "Gold", True)).alias("Total_No_Of_Gold_Medals"),
    count(when(col("src.Medal") == "Silver", True)).alias("Total_No_Of_Silver_Medals"),
    count(when(col("src.Medal") == "Bronze", True)).alias("Total_No_Of_Bronze_Medals"),
    countDistinct(when(col("src.Player_Gender") == "F", col("src.Player_Name"))).alias("No_Of_Women_Players"),
    countDistinct(when(col("src.Player_Gender") == "M", col("src.Player_Name"))).alias("No_Of_Men_Players"),
    countDistinct("src.Player_Name").alias("No_Of_Players"),
    countDistinct(when(col("src.Medal").isNotNull(), col("src.Player_Name"))).alias("No_Of_Players_Won"),
    countDistinct(when(col("src.Medal").isNull(), col("src.Player_Name"))).alias("No_Of_Players_Lost")
)
fact_country_details = fact_country_details.withColumn(
    "Male_Female_Ratio",
    round(col("No_Of_Men_Players") / (col("No_Of_Women_Players") + 1), 2)
)

fact_country_details.write.mode("overwrite").csv("abfss://facts@hexaolympicdl.dfs.core.windows.net/Olympic_facts")

Country_ID Olympic_ID Olympic_Year Country_Name Total_No_Of_Medals Total_No_Of_Gold_Medals Total_No_Of_Silver_Medals Total_No_Of_Bronze_Medals No_Of_Women_Players No_Of_Men_Players No_Of_Players No_Of_Players_Won No_Of_Players_Lost Male_Female_Ratio SEY 5 2016 Seychelles 10 0 0 0 2 8 10 10 0 2.67 UKR 5 2016 Ukraine 268 2 8 5 114 87 201 201 0 0.76 RUS 3 2012 Russia 541 50 39 49 221 203 424 424 0 0.91 SOL 1 2008 Solomon Islands 3 0 0 0 2 1 3 3 0 0.33 GER 1 2008 Elvis Va 1 1 0 0 1 0 1 1 0 0.0 BAN 1 2008 Bangladesh 5 0 0 0 2 3 5 5 0 1.0 CHN 5 2016 China-2 8 0 0 0 5 3 8 8 0 0.5 INA 5 2016 Indonesia-1 2 2 0 0 1 1 2 2 0 0.5 IND 4 2014 India 2 0 0 0 0 2 2 2 0 2.0 ISL 2 2010 Iceland 7 0 0 0 1 3 4 4 0 1.5

In [0]:
dummydf=fact_country_details.groupBy("Olympic_Year","Olympic_ID").count()
dummydfalias=dummydf.alias("d")
z=dim_olympic_alias.join(dummydfalias,on=col("d.Olympic_ID")==col("o.Olympic_ID"),how="left").select("d.Olympic_Year","o.Olympic_Name","d.count")
z.display()
z.write.mode("overwrite").csv("abfss://report@hexaolympicdl.dfs.core.windows.net/transformation1")



Olympic_Year Olympic_Name count 2008 2008 Summer 292 2012 2012 Summer 245 2016 2016 Summer 249 2014 2014 Winter 119 2010 2010 Winter 116

In [0]:
dummydf=fact_country_details.groupBy("Olympic_Year","Olympic_ID").agg(countDistinct(col("Country_ID")))
#dummydf.display()
dummydfalias=dummydf.alias("d")
z1=dim_olympic_alias.join(dummydfalias,on=col("d.Olympic_ID")==col("o.Olympic_ID"),how="left").select("d.Olympic_Year","o.Olympic_Name","d.count(DISTINCT Country_ID)")
z1.display()
z1.write.mode("overwrite").csv("abfss://report@hexaolympicdl.dfs.core.windows.net/transformation2")

Olympic_Year Olympic_Name count(DISTINCT Country_ID) 2008 2008 Summer 204 2012 2012 Summer 205 2016 2016 Summer 207 2014 2014 Winter 89 2010 2010 Winter 82

In [0]:
z2=fact_country_details.groupBy("Olympic_Year","Country_Name").agg({"total_No_Of_Gold_Medals":"sum"}).filter("Olympic_Year==2016").orderBy("sum(total_No_Of_Gold_Medals)",ascending=False).limit(10)
z2.display()
z2.write.mode("overwrite").csv("abfss://report@hexaolympicdl.dfs.core.windows.net/transformation3")

Olympic_Year Country_Name sum(total_No_Of_Gold_Medals) 2016 United States 137 2016 Great Britain 64 2016 Russia 50 2016 Germany 47 2016 China 44 2016 Brazil 34 2016 Australia 23 2016 Argentina 21 2016 France 20 2016 Japan 17

In [0]:
from pyspark.sql.functions import avg
z3=fact_country_details.groupBy("Olympic_Year","Country_Name").agg(avg("Male_Female_Ratio")).orderBy("Olympic_Year","Country_Name").filter( (col("Country_Name")=="China"))
z3.display()
z3.write.mode("overwrite").csv("abfss://report@hexaolympicdl.dfs.core.windows.net/transformation4")

Olympic_Year Country_Name avg(Male_Female_Ratio) 2008 China 1.09 2010 China 0.51 2012 China 0.75 2014 China 0.69 2016 China 0.62

In [0]:
ab=fact_df.groupBy("s.Sport_ID","p.Player_Name").agg({"Medal_ID":"sum"}).filter(col("Sport_ID")==41).orderBy(col("sum(Medal_ID)").desc()).limit(3)
abalias=ab.alias("ab")
res=abalias.join(dim_sport_alias,on=col("s.Sport_ID")==col("ab.Sport_ID"),how="left").select("Player_Name","Sport_Name","sum(Medal_ID)").withColumnRenamed("sum(Medal_ID)","Total_Medals")
res.display()
res.write.mode("overwrite").csv("abfss://report@hexaolympicdl.dfs.core.windows.net/transformation5")

Player_Name Sport_Name Total_Medals Michael Fred Phelps, II Swimming 49 Sarah Frederica Sjstrm Swimming 46 Mireia Belmonte Garca Swimming 44